In [14]:
# Setup: Add package to path (for development mode)
import sys
sys.path.insert(0, '/Users/aakashroy/Downloads/or-af framework development/or-af')

import or_af
print(f"OR-AF v{or_af.__version__} loaded successfully!")

OR-AF v0.4.0 loaded successfully!


# OR-AF Framework - Complete Feature Guide

This notebook demonstrates all features of OR-AF (Operations Research Agentic Framework):

| # | Feature | Description |
|---|---------|-------------|
| 1 | **MCP Servers** | Create tool servers using the MCP protocol |
| 2 | **Agents** | AI agents that connect to MCP servers |
| 3 | **Workflows** | Build agent pipelines with graphs |
| 4 | **A2A Protocol** | Agent-to-agent communication |
| 5 | **Callbacks** | Monitor agent execution |
| 6 | **Logging** | Built-in logging utilities |

## 1. MCP Servers - Host Your Tools

MCP (Model Context Protocol) servers host tools that agents can use. Create a server and register tools with decorators.

In [15]:
from or_af import create_mcp_server

# Create an MCP server
math_server = create_mcp_server(name="math_tools", description="Math operations")

# Register tools using @server.tool() decorator
@math_server.tool()
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

@math_server.tool()
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

print(f"✓ MCP Server: {math_server.name}")
print(f"  Tools: {math_server.list_tools()}")

23:07:32 | INFO | MCP Server 'math_tools' initialized (using official MCP SDK)


[01/31/26 23:07:32] INFO     MCP Server 'math_tools' initialized (using official MCP SDK)              logger.py:65

23:07:32 | INFO | Tool 'add' registered with MCP server 'math_tools'


                    INFO     Tool 'add' registered with MCP server 'math_tools'                        logger.py:65

23:07:32 | INFO | Tool 'multiply' registered with MCP server 'math_tools'


                    INFO     Tool 'multiply' registered with MCP server 'math_tools'                   logger.py:65

✓ MCP Server: math_tools
  Tools: ['add', 'multiply']


## 2. Agents - Connect to MCP Servers

Agents connect to MCP servers to access tools. They use tools to accomplish tasks.

In [3]:
from or_af import AgentConfig

# Note: Agent requires Azure OpenAI credentials in environment variables.
# For demo purposes, we'll show the configuration:

config = AgentConfig(
    system_prompt="You are a calculator. Use tools to compute results.",
    max_iterations=5,
    temperature=0.7
)

print(f"✓ AgentConfig created")
print(f"  System prompt: {config.system_prompt[:40]}...")
print(f"  Max iterations: {config.max_iterations}")
print(f"  Temperature: {config.temperature}")

print("\n📝 To create a full Agent (requires Azure OpenAI credentials):")
print('''
agent = Agent(
    name="calculator",
    system_prompt="...",
    mcp_servers=[math_server]
)
''')

✓ AgentConfig created
  System prompt: You are a calculator. Use tools to compu...
  Max iterations: 5
  Temperature: 0.7

📝 To create a full Agent (requires Azure OpenAI credentials):

agent = Agent(
    name="calculator",
    system_prompt="...",
    mcp_servers=[math_server]
)



## 3. Workflows - Build Agent Pipelines

Use `WorkflowGraph` to create multi-agent pipelines with conditional routing.

In [4]:
from or_af import WorkflowGraph, EdgeCondition

# Create simple processing functions for the workflow demo
def research_func(input_data):
    return f"Researched: {input_data}"

def write_func(input_data):
    return f"Written report on: {input_data}"

# Build a workflow graph
wf = WorkflowGraph(name="demo_pipeline")
node1 = wf.add_node(research_func, name="Research", is_entry=True)
node2 = wf.add_node(write_func, name="Write", is_exit=True)
wf.add_edge(node1, node2, condition=EdgeCondition.ON_SUCCESS)
wf.compile()

print(f"✓ Workflow: {wf.name}")
print(f"  Nodes: {len(wf.nodes)}, Edges: {len(wf.edges)}")

23:01:46 | INFO | Node 'Research' added to workflow 'demo_pipeline'


[01/31/26 23:01:46] INFO     Node 'Research' added to workflow 'demo_pipeline'                         logger.py:65

23:01:46 | INFO | Node 'Write' added to workflow 'demo_pipeline'


                    INFO     Node 'Write' added to workflow 'demo_pipeline'                            logger.py:65

23:01:46 | INFO | Edge 'Research->Write' added to workflow


                    INFO     Edge 'Research->Write' added to workflow                                  logger.py:65

23:01:46 | INFO | Workflow 'demo_pipeline' compiled successfully


                    INFO     Workflow 'demo_pipeline' compiled successfully                            logger.py:65

✓ Workflow: demo_pipeline
  Nodes: 2, Edges: 1


### Workflow Visualization

OR-AF supports multiple visualization formats: text, ASCII, and Mermaid diagrams.

In [5]:
# Text visualization
print("TEXT FORMAT:")
print(wf.visualize(format="text"))

# ASCII visualization  
print("\nASCII FORMAT:")
print(wf.visualize(format="ascii"))

TEXT FORMAT:
📊 Workflow: demo_pipeline
🔢 Nodes: 2 | Edges: 1

🔷 NODES:
----------------------------------------
  ⏳ Research 🚀[ENTRY]
  ⏳ Write 🏁[EXIT]

🔗 EDGES:
----------------------------------------
  Research ✅──▶ Write
      Condition: on_success

ASCII FORMAT:
+==========================================================+
|                      demo_pipeline                       |
+==========================================================+

+------------------+
| Research [ENTRY] |
+------------------+
     |
     | (on_success)
     v
+------------------+
|   Write [EXIT]   |
+------------------+


### TensorFlow-like Sequential API

Use `Sequential` for linear workflows - similar to `tf.keras.Sequential`.

In [6]:
from or_af import Sequential

# Define simple processing steps
step1 = lambda x: f"[Step1] {x}"
step2 = lambda x: f"[Step2] {x}"
step3 = lambda x: f"[Step3] {x}"

# Create sequential workflow
seq = Sequential([step1, step2, step3], name="pipeline")
print(f"✓ Sequential: {seq.name}")
print(seq.visualize(format="text"))

23:01:49 | INFO | Node 'agent_0' added to workflow 'pipeline'


[01/31/26 23:01:49] INFO     Node 'agent_0' added to workflow 'pipeline'                               logger.py:65

23:01:49 | INFO | Node 'agent_1' added to workflow 'pipeline'


                    INFO     Node 'agent_1' added to workflow 'pipeline'                               logger.py:65

23:01:49 | INFO | Node 'agent_2' added to workflow 'pipeline'


...

23:01:49 | INFO | Edge 'agent_1->agent_2' added to workflow


                    INFO     Edge 'agent_1->agent_2' added to workflow                                 logger.py:65

23:01:49 | INFO | Workflow 'pipeline' compiled successfully


                    INFO     Workflow 'pipeline' compiled successfully                                 logger.py:65

✓ Sequential: pipeline
📊 Workflow: pipeline
🔢 Nodes: 3 | Edges: 2

🔷 NODES:
----------------------------------------
  ⏳ agent_0 🚀[ENTRY]
  ⏳ agent_1
  ⏳ agent_2 🏁[EXIT]

🔗 EDGES:
----------------------------------------
  agent_0 ✅──▶ agent_1
      Condition: on_success
  agent_1 ✅──▶ agent_2
      Condition: on_success


## 4. A2A Protocol - Agent Communication

A2A (Agent-to-Agent) protocol enables agents to communicate with each other.

In [7]:
from or_af import create_a2a_agent, SimpleA2AExecutor

# Define a handler for the A2A agent
async def echo_handler(message: str) -> str:
    return f"Echo: {message}"

# Create an A2A-compliant agent
a2a_agent = create_a2a_agent(
    name="EchoBot",
    description="A simple echo agent",
    skills=[{"id": "echo", "name": "Echo", "description": "Echoes messages", "tags": ["echo"]}]
)
a2a_agent.set_executor(SimpleA2AExecutor(echo_handler))

print(f"✓ A2A Agent: {a2a_agent.name}")
print(f"  Skills: {[s.name for s in a2a_agent.card.skills]}")

23:01:50 | INFO | A2A Agent 'EchoBot' initialized (using official A2A SDK)


[01/31/26 23:01:50] INFO     A2A Agent 'EchoBot' initialized (using official A2A SDK)                  logger.py:65

23:01:50 | INFO | Executor set for A2A Agent 'EchoBot'


                    INFO     Executor set for A2A Agent 'EchoBot'                                      logger.py:65

✓ A2A Agent: EchoBot
  Skills: ['Echo']


## 5. Callbacks - Monitor Agent Execution

Callbacks let you track what happens during agent execution.

In [8]:
from or_af import ConsoleCallback, MetricsCallback, CallbackHandler, EventType

# Available callbacks
console_cb = ConsoleCallback()    # Prints events to console
metrics_cb = MetricsCallback()    # Tracks execution metrics

# Callback handler manages callbacks by event type
handler = CallbackHandler()
handler.register_global(console_cb.on_event)  # Register for all events
handler.register(EventType.AGENT_START, metrics_cb.on_event)

print(f"✓ Callbacks: ConsoleCallback, MetricsCallback")
print(f"  Global callbacks: {len(handler._global_callbacks)}")
print(f"  Event types with callbacks: {sum(1 for v in handler._callbacks.values() if v)}")

✓ Callbacks: ConsoleCallback, MetricsCallback
  Global callbacks: 1
  Event types with callbacks: 1


## 6. Logging - Built-in Utilities

OR-AF includes a configurable logging system.

In [9]:
from or_af import get_logger, set_log_level, LogLevel

# Get a logger
logger = get_logger("my_app")

# Set log level (DEBUG, INFO, WARNING, ERROR)
set_log_level(LogLevel.INFO)

logger.info("This is an info message")
logger.warning("This is a warning")

print(f"✓ Logger configured with level: INFO")

23:01:53 | INFO | This is an info message


[01/31/26 23:01:53] INFO     This is an info message                                                   logger.py:65

23:01:53 | WARNING | This is a warning


                    WARNING  This is a warning                                                         logger.py:69

✓ Logger configured with level: INFO


## 7. Models - Data Structures

OR-AF provides Pydantic models for type-safe data handling.

In [10]:
from or_af import AgentConfig, ToolCall, ToolResult, Message, MessageRole

# AgentConfig - Configure agent behavior
config = AgentConfig(system_prompt="Be helpful", max_iterations=5)

# Message - Chat messages
msg = Message(role=MessageRole.USER, content="Hello!")

# ToolCall - Track tool invocations
tool_call = ToolCall(id="call_123", name="add", arguments={"a": 1, "b": 2})

# ToolResult - Track results
tool_result = ToolResult(
    tool_call_id="call_123", 
    tool_name="add", 
    result=3, 
    execution_time=0.01
)

print(f"✓ Models available:")
print(f"  AgentConfig: system_prompt='{config.system_prompt}'")
print(f"  Message: {msg.role} - '{msg.content}'")
print(f"  ToolCall: {tool_call.name}({tool_call.arguments})")
print(f"  ToolResult: {tool_result.tool_name} = {tool_result.result}")

✓ Models available:
  AgentConfig: system_prompt='Be helpful'
  Message: MessageRole.USER - 'Hello!'
  ToolCall: add({'a': 1, 'b': 2})
  ToolResult: add = 3


## 8. Exceptions - Error Handling

OR-AF provides specific exceptions for different error types.

In [11]:
from or_af import (
    ORAFError,           # Base exception
    AgentError,          # Agent-related errors
    ToolError,           # Tool errors
    MCPError,            # MCP server errors
    WorkflowError,       # Workflow errors
    A2AError             # A2A protocol errors
)

# Example: Handle errors gracefully
try:
    raise ToolError("Tool not found: unknown_tool")
except ORAFError as e:
    print(f"✓ Caught OR-AF error: {e}")

print(f"\n✓ Exception hierarchy:")
print(f"  ORAFError (base)")
print(f"    ├── AgentError")
print(f"    ├── ToolError")
print(f"    ├── MCPError")
print(f"    ├── WorkflowError")
print(f"    └── A2AError")

✓ Caught OR-AF error: Tool not found: unknown_tool

✓ Exception hierarchy:
  ORAFError (base)
    ├── AgentError
    ├── ToolError
    ├── MCPError
    ├── WorkflowError
    └── A2AError


## Summary

| Feature | Key Classes | Description |
|---------|-------------|-------------|
| **MCP** | `MCPServer`, `create_mcp_server` | Host tools on servers |
| **Agents** | `Agent` | Connect to MCP servers, run tasks |
| **Workflows** | `WorkflowGraph`, `Sequential`, `Parallel` | Build agent pipelines |
| **A2A** | `A2AAgent`, `create_a2a_agent` | Agent-to-agent communication |
| **Callbacks** | `ConsoleCallback`, `MetricsCallback` | Monitor execution |
| **Logging** | `get_logger`, `set_log_level` | Built-in logging |
| **Models** | `AgentConfig`, `Message`, `ToolCall` | Type-safe data |
| **Exceptions** | `ORAFError`, `AgentError`, etc. | Error handling |

**Documentation:** See README.md and QUICKSTART.md for more details.

## Testing Real LLM Execution

Now let's test if the LLM integration is working properly with a real agent execution.

In [16]:
from or_af import Agent
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify environment variables are loaded
print("Environment check:")
has_endpoint = bool(os.getenv('AZURE_OPENAI_ENDPOINT'))
has_key = bool(os.getenv('AZURE_OPENAI_API_KEY'))
has_deployment = bool(os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'))

print(f"  AZURE_OPENAI_ENDPOINT: {'✓' if has_endpoint else '✗ (using default)'}")
print(f"  AZURE_OPENAI_API_KEY: {'✓' if has_key else '✗ (using default)'}")
print(f"  AZURE_OPENAI_DEPLOYMENT_NAME: {'✓' if has_deployment else '✗ (using default)'}")

# Create a real agent with the math server
calculator_agent = Agent(
    name="calculator",
    system_prompt="You are a helpful calculator assistant. Use the available tools to perform calculations.",
    mcp_servers=[math_server],
    max_iterations=5
)

print(f"\n✓ Agent created: {calculator_agent.name}")
print(f"  Available tools: {calculator_agent.list_available_tools()}")

Environment check:
  AZURE_OPENAI_ENDPOINT: ✓
  AZURE_OPENAI_API_KEY: ✓
  AZURE_OPENAI_DEPLOYMENT_NAME: ✓
23:07:41 | INFO | Agent 'calculator' connected to MCP server 'math_tools'


[01/31/26 23:07:41] INFO     Agent 'calculator' connected to MCP server 'math_tools'                   logger.py:65

✓ Connected to MCP server 'math_tools' (2 tools)
23:07:41 | INFO | Agent 'calculator' initialized with model: gpt-5


                    INFO     Agent 'calculator' initialized with model: gpt-5                          logger.py:65


✓ Agent created: calculator
  Available tools: {'math_tools': ['add', 'multiply']}


In [17]:
# Test the agent with a calculation task
task = "Calculate 15 multiplied by 23, then add 100 to the result"

print(f"Task: {task}\n")
print("Running agent...\n")

# Run the agent
result = calculator_agent.run(task)

print(f"\n✓ Agent completed!")
print(f"Final result: {result}")

Task: Calculate 15 multiplied by 23, then add 100 to the result

Running agent...

23:07:49 🚀 agent_start 
23:07:49 | INFO | Agent started with task: Calculate 15 multiplied by 23, then add 100 to the result


[01/31/26 23:07:49] INFO     Agent started with task: Calculate 15 multiplied by 23, then add 100 to   logger.py:65
                             the result                                                                            

23:07:49 🔄 iteration_start [1]


[01/31/26 23:07:56] INFO     HTTP Request: POST                                                     _client.py:1025
                             https://bimigration.openai.azure.com/openai/deployments/gpt-5/chat/com                
                             pletions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"                             

23:07:56 📝 stream_chunk 
23:07:56 💭 thinking [1]
23:07:56 ✓ iteration_end [1]
23:07:56 | INFO | Task completed in 1 iteration(s)


                    INFO     Task completed in 1 iteration(s)                                          logger.py:65

23:07:56 🏁 agent_end 
23:07:56 | INFO | Agent finished. Success: True, Duration: 6.61s


                    INFO     Agent finished. Success: True, Duration: 6.61s                            logger.py:65


✓ Agent completed!
Final result: task='Calculate 15 multiplied by 23, then add 100 to the result' response='445' iterations=[IterationState(iteration_number=1, tool_calls=[], tool_results=[], thinking='445', response='445', start_time=datetime.datetime(2026, 1, 31, 23, 7, 49, 837535), end_time=datetime.datetime(2026, 1, 31, 23, 7, 56, 442127))] total_tool_calls=0 success=True error_message=None start_time=datetime.datetime(2026, 1, 31, 23, 7, 49, 836118) end_time=datetime.datetime(2026, 1, 31, 23, 7, 56, 447855)


## Advanced Workflow with Graph Visualization

Let's create a more complex workflow with multiple agents and visualize the graph structure.

In [19]:
# Create a multi-agent workflow
from or_af import WorkflowGraph, EdgeCondition

# Create different agent functions for the workflow
def research_agent(task):
    """Simulates a research agent"""
    print(f"🔍 Research Agent: Gathering information on '{task}'")
    return f"Research data for: {task}"

def analysis_agent(data):
    """Simulates an analysis agent"""
    print(f"📊 Analysis Agent: Analyzing data")
    return f"Analysis of: {data}"

def writer_agent(analysis):
    """Simulates a writer agent"""
    print(f"✍️  Writer Agent: Creating report")
    return f"Report: {analysis}"

def reviewer_agent(report):
    """Simulates a reviewer agent"""
    print(f"👁️  Reviewer Agent: Reviewing report")
    return f"Reviewed: {report}"

# Build a complex workflow graph
complex_wf = WorkflowGraph(name="multi_agent_pipeline")

# Add nodes
research = complex_wf.add_node(research_agent, name="Research", is_entry=True)
analysis = complex_wf.add_node(analysis_agent, name="Analysis")
writer = complex_wf.add_node(writer_agent, name="Writer")
reviewer = complex_wf.add_node(reviewer_agent, name="Reviewer", is_exit=True)

# Add edges with conditions
complex_wf.add_edge(research, analysis, condition=EdgeCondition.ON_SUCCESS)
complex_wf.add_edge(analysis, writer, condition=EdgeCondition.ON_SUCCESS)
complex_wf.add_edge(writer, reviewer, condition=EdgeCondition.ON_SUCCESS)

# Compile the workflow
complex_wf.compile()

print(f"✓ Complex Workflow created: {complex_wf.name}")
print(f"  Nodes: {len(complex_wf.nodes)}")
print(f"  Edges: {len(complex_wf.edges)}")
print(f"  Entry node: {complex_wf.entry_node.name if complex_wf.entry_node else 'None'}")
print(f"  Exit nodes: {[complex_wf.nodes[n].name for n in complex_wf.exit_nodes]}")

23:08:29 | INFO | Node 'Research' added to workflow 'multi_agent_pipeline'


[01/31/26 23:08:29] INFO     Node 'Research' added to workflow 'multi_agent_pipeline'                  logger.py:65

23:08:29 | INFO | Node 'Analysis' added to workflow 'multi_agent_pipeline'


                    INFO     Node 'Analysis' added to workflow 'multi_agent_pipeline'                  logger.py:65

23:08:29 | INFO | Node 'Writer' added to workflow 'multi_agent_pipeline'


...

23:08:29 | INFO | Edge 'Writer->Reviewer' added to workflow


                    INFO     Edge 'Writer->Reviewer' added to workflow                                 logger.py:65

23:08:29 | INFO | Workflow 'multi_agent_pipeline' compiled successfully


                    INFO     Workflow 'multi_agent_pipeline' compiled successfully                     logger.py:65

✓ Complex Workflow created: multi_agent_pipeline
  Nodes: 4
  Edges: 3
  Entry node: Research
  Exit nodes: ['Reviewer']


### Workflow Graph Visualization

**This is a key feature of OR-AF!** The framework provides multiple visualization formats to help you understand your workflow structure.

In [20]:
print("=" * 60)
print("TEXT VISUALIZATION")
print("=" * 60)
print(complex_wf.visualize(format="text"))

print("\n" + "=" * 60)
print("ASCII GRAPH VISUALIZATION")
print("=" * 60)
print(complex_wf.visualize(format="ascii"))

print("\n" + "=" * 60)
print("MERMAID DIAGRAM (for documentation)")
print("=" * 60)
print(complex_wf.visualize(format="mermaid"))

TEXT VISUALIZATION
📊 Workflow: multi_agent_pipeline
🔢 Nodes: 4 | Edges: 3

🔷 NODES:
----------------------------------------
  ⏳ Research 🚀[ENTRY]
  ⏳ Analysis
  ⏳ Writer
  ⏳ Reviewer 🏁[EXIT]

🔗 EDGES:
----------------------------------------
  Research ✅──▶ Analysis
      Condition: on_success
  Analysis ✅──▶ Writer
      Condition: on_success
  Writer ✅──▶ Reviewer
      Condition: on_success

ASCII GRAPH VISUALIZATION
+==========================================================+
|                   multi_agent_pipeline                   |
+==========================================================+

+------------------+
| Research [ENTRY] |
+------------------+
     |
     | (on_success)
     v
+------------------+
|     Analysis     |
+------------------+
     |
     | (on_success)
     v
+------------------+
|      Writer      |
+------------------+
     |
     | (on_success)
     v
+------------------+
| Reviewer [EXIT]  |
+------------------+

MERMAID DIAGRAM (for documentation)


In [21]:
# Execute the workflow
print("\n" + "=" * 60)
print("EXECUTING WORKFLOW")
print("=" * 60)

result = complex_wf.run("AI and Machine Learning trends")

print(f"\n✓ Workflow execution complete!")
print(f"Final result: {result}")


EXECUTING WORKFLOW
23:08:43 | INFO | Executing node: Research


[01/31/26 23:08:43] INFO     Executing node: Research                                                  logger.py:65

🔍 Research Agent: Gathering information on 'AI and Machine Learning trends'
23:08:43 | INFO | Executing node: Analysis


                    INFO     Executing node: Analysis                                                  logger.py:65

📊 Analysis Agent: Analyzing data
23:08:43 | INFO | Executing node: Writer


...

✍️  Writer Agent: Creating report
23:08:43 | INFO | Executing node: Reviewer


                    INFO     Executing node: Reviewer                                                  logger.py:65

👁️  Reviewer Agent: Reviewing report
23:08:43 | INFO | Workflow 'multi_agent_pipeline' execution completed


                    INFO     Workflow 'multi_agent_pipeline' execution completed                       logger.py:65


✓ Workflow execution complete!
Final result: {'workflow_id': '74c15e23-0cad-4182-89dd-d0e2b9e2e3bf', 'workflow_name': 'multi_agent_pipeline', 'input': 'AI and Machine Learning trends', 'results': {'71903b4d-11db-49b7-942e-16f70af06fe1': NodeResult(node_id='71903b4d-11db-49b7-942e-16f70af06fe1', status=<NodeStatus.COMPLETED: 'completed'>, output='Research data for: AI and Machine Learning trends', error=None, execution_time=2.193450927734375e-05, timestamp=datetime.datetime(2026, 1, 31, 23, 8, 43, 963857)), '77d8b3fd-3729-45ba-9ad3-c45076683043': NodeResult(node_id='77d8b3fd-3729-45ba-9ad3-c45076683043', status=<NodeStatus.COMPLETED: 'completed'>, output='Analysis of: Research data for: AI and Machine Learning trends', error=None, execution_time=1.5020370483398438e-05, timestamp=datetime.datetime(2026, 1, 31, 23, 8, 43, 964882)), '7d6861fd-cd2b-4e7c-a0c3-3aaf65905d7e': NodeResult(node_id='7d6861fd-cd2b-4e7c-a0c3-3aaf65905d7e', status=<NodeStatus.COMPLETED: 'completed'>, output='Report: